# Part I. ETL Pipeline for Pre-Processing the Files

## PRE-PROCESSING THE FILES

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/algo_r3x/DEND/apache_cassandra_breakingice_g


#### Processing the files to create a data file csv that will be used for read and insert into Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:
    
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
              
        for line in csvreader:
            full_data_rows_list.append(line) 
            

# creating the smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))
    

6821


# Part II. Apache Cassandra

##  <font color=red>event_datafile_new.csv</font>,  columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId


In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

# Create session
session = cluster.connect()

In [6]:
#Keyspace Sparkify
query = """
CREATE KEYSPACE IF NOT EXISTS sparkify
WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1} 
"""

session.execute(query)

session.set_keyspace('sparkify')

### Table Creation - based on the queries that need to be run

### Query 1

In [7]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# Primary key - sessionId
# clustering column - itemInSession

query = """
CREATE TABLE IF NOT EXISTS track_plays
(artist varchar, track_title varchar, track_length float, session_id int, item_in_session int, PRIMARY KEY (session_id, item_in_session))
"""
session.execute(query)

In [8]:
# Process csv file and insert rows into table 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO track_plays (artist, track_title, track_length, session_id, item_in_session)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"

        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

# Verify the data was entered into the table
query = """ SELECT artist, track_title, track_length FROM track_plays where session_id=338 and item_in_session=4 """
rows = session.execute(query)

df = pd.DataFrame(list(rows))
df

,artist,track_title,track_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query 2

In [9]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# Primary key - user_id, session_id
# Clustering column - item_in_session  --> we need result in order by desc

query = """
CREATE TABLE IF NOT EXISTS user_sessions_plays(
user_id int, 
session_id int,
item_in_session int,
user varchar,
artist varchar,
track_title varchar,
PRIMARY KEY ((user_id, session_id), item_in_session)
)
"""
session.execute(query)

In [10]:
# Process csv file and insert rows into table 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO user_sessions_plays (user_id, session_id, item_in_session, user, artist, track_title)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"

        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1]+' '+line[4], line[0], line[9]))
        
query = """ SELECT artist, track_title, user from user_sessions_plays where user_id=10 and session_id=182"""
rows = session.execute(query)
df = pd.DataFrame(list(rows))
df

,artist,track_title,user
0,Down To The Bone,Keep On Keepin' On,Sylvie Cruz
1,Three Drives,Greece 2000,Sylvie Cruz
2,Sebastien Tellier,Kilometer,Sylvie Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie Cruz


### Query 3

In [11]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#Primary Key - track_title, user_id

query = """ 
CREATE TABLE IF NOT EXISTS user_track_plays(
track_title varchar, 
user_id int, 
user varchar,
PRIMARY KEY (track_title, user_id)
)
"""
session.execute(query)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO user_track_plays (track_title, user_id, user)"
        query = query + " VALUES (%s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1]+' '+line[4]))

query = """ SELECT user FROM user_track_plays where track_title='All Hands Against His Own' """
rows = session.execute(query)
df = pd.DataFrame(list(rows))
df

,user
0,Jacqueline Lynch
1,Tegan Levine
2,Sara Johnson


In [13]:
#: Drop the tables. Close connection. Shtudown cluster

In [14]:
queries =[ 
"""DROP TABLE IF EXISTS track_plays ; """,
"""DROP TABLE IF EXISTS user_session_plays ; """,
"""DROP TABLE IF EXISTS user_track_plays  """
]
for query in queries:
    session.execute(query)

In [15]:
session.shutdown()
cluster.shutdown()